In [16]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $400000$) and make a hash table from the $a_p$ for $30$ primes (the first $30$ over $400000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [2]:
try:
    hashtab_11_30 = load('hashtab_11_30')
except IOError:
    hashtab_11_30 = make_hash(11,1,400000,30)
    save(hashtab_11_30, 'hashtab_11_30')

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{11}$ for all $30$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [3]:
congruent_sets = [s for s in hashtab_11_30.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 636
Distribution of sizes of nontrivial congruent sets: Counter({2: 636})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors).  With the default mumax=5000000 there is one pair for which this is not rigorous:  '49a1' and '78400gw1'.

In [4]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(11,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,11,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 11-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 11-congruent".format(len(congruent_sets)))

Curves 49a1 and 78400gw1: testing ell up to 14837760 mod 11
Congruence mod 11 fails for 25921a1 and 78400gw1
 (not even isomorphic up to semisimplication: (29, 2, -2) )
1 pairs of curves are not actually 11-congruent: [['25921a1', '78400gw1']]


### So we have one "false positive" pair which we now remove from the list of congruent pairs:

In [5]:
bad_pair = bad_pairs[0]
assert bad_pair in congruent_sets
congruent_sets.remove(bad_pair)
print("After removing the false positive pair {}, there are {} congruent pairs remaining".format(bad_pair,len(congruent_sets)))

After removing the false positive pair ['25921a1', '78400gw1'], there are 635 congruent pairs remaining


### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [6]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 635 pairs, there are only 89 distinct pairs of j-invariants


In [7]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

89 congruent pairs up to twist: [['30576ct1', '336336hg1'], ['31920bg1', '31920bh1'], ['78650cy1', '78650dj1'], ['29106g1', '378378bf1'], ['256880cx1', '256880cy1'], ['1935h1', '210915j1'], ['1254b1', '264594e1'], ['95550fc1', '95550fd1'], ['2352r1', '39984cw1'], ['307626j1', '307626k1'], ['187690d1', '187690e1'], ['414a1', '52578j1'], ['8640m1', '319680em1'], ['263578b1', '263578c1'], ['221130cd1', '221130cf1'], ['8120d1', '332920k1'], ['80325da1', '80325db1'], ['208080u1', '208080bb1'], ['1115a1', '125995a1'], ['338800dc1', '338800dd1'], ['2093b1', '26611d1'], ['32856o1', '229992e1'], ['144150cp1', '144150cq1'], ['63504t1', '63504u1'], ['31746q1', '222222bb1'], ['11700w1', '198900h1'], ['10082c1', '70574h1'], ['84672jo1', '84672jp1'], ['202400r1', '349600y1'], ['513a1', '77463a1'], ['4275d1', '175275k1'], ['304704bn1', '304704bs1'], ['5454b1', '59994e1'], ['5950p1', '113050cj1'], ['3718s1', '107822o1'], ['17550t1', '298350bc1'], ['53290m1', '53290n1'], ['255486c1', '394842d1'], ['284

### Test whether all these are irreducible, i.e. have no $11$-isogenies:

In [8]:
for s2 in congruent_pairs:
    for s in s2:
        E = EllipticCurve(s)
        assert not E.isogenies_prime_degree(11)

#### In fact the only isogeny degrees among these curves are 2, 3, 7:

In [9]:
sum([Set(EllipticCurve(s[0]).reducible_primes()) for s in congruent_pairs],Set())

{2, 3, 7}

In [58]:
out = open("mod11pairs_uptotwist.m",'w')
out.write('pairs := [\\\n')
for s in congruent_pairs[:-1]:
    out.write('[EllipticCurve("{}"), EllipticCurve("{}")],\\\n'.format(s[0],s[1]))
s = congruent_pairs[-1]
out.write('[EllipticCurve("{}"), EllipticCurve("{}")]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [10]:
len(congruent_pairs)

89

In [13]:
for pair in congruent_pairs:
    sizes = [len(EllipticCurve(lab).isogeny_class()) for lab in pair]
    if sizes != [1,1]:
        print("pair {}: class sizes {}".format(pair,sizes))

pair ['78650cy1', '78650dj1']: class sizes [2, 1]
pair ['2352r1', '39984cw1']: class sizes [2, 1]
pair ['414a1', '52578j1']: class sizes [4, 1]
pair ['4275d1', '175275k1']: class sizes [2, 1]
pair ['5950p1', '113050cj1']: class sizes [2, 1]
pair ['53290m1', '53290n1']: class sizes [2, 1]
pair ['12600cl1', '239400cw1']: class sizes [2, 1]
pair ['3850i1', '65450i1']: class sizes [2, 1]
pair ['426b1', '77106c1']: class sizes [2, 1]
pair ['5200bh1', '369200w1']: class sizes [2, 1]
pair ['193358bf1', '193358bg1']: class sizes [2, 1]
pair ['11200bb1', '190400dh1']: class sizes [2, 1]


In [15]:
[legendre_symbol(a,11) for a in [2,3,7]]

[-1, 1, -1]

In [26]:
npc=0
out = open("mod11_pairs.m", "w")
out.write('pairs := [\\\n')
for i,pair in enumerate(congruent_pairs):
    cc = [all_c(c) for c in pair]
    for c1 in cc[0]:
        for c2 in cc[1]:
            npc+=1
            out.write('["{}", "{}"]'.format(c1[0],c2[0]))
            if i+1<len(congruent_pairs):
                out.write(',')
            out.write('\\\n')
out.write('];\n')
out.close()           
print("{} pairs written to file".format(npc))            

103 pairs written to file


In [28]:
len(congruent_pairs)

89